# Homework 8: SQL and PyTorch(21 pts)

name: Zhekai Liu

email: zhekail@umich.edu

This homework assignment took me 15 hours in total to complete. (Please help us to gauge the difficulty of the assignment.)

## Collaboration Disclosure

In the cell below, please list *everyone* with whom you discussed any of the homework problems, excluding only the GSIs and the course instructor. 

If you did not discuss the homework with anyone else, write __"I did not discuss this homework with anyone."__

Even if you discuss questions with other, the code you submit must be only yours. All work is checked with the [MOSS plagiarism detector](https://theory.stanford.edu/~aiken/moss/).

I did not discuss this homework with anyone.

## Submission Instructions
Your homework solutions should be written entirely in this Jupyter notebook file. Once it contains your solutions, you should submit this notebook through Canvas. 


Before submitting, please make sure to __Cells->Run All__ executes without errors; errors in your code translate directly to point deductions. 
In general, you don't need to do explicitly raise errors (e.g. with the ```raise``` function) if we don't ask you to in the problem statement.
However, even in cases where we ask you to check for errors, your submission should not contain any examples of your functions actually raising those errors.

Note that many parts of this homework where you are expected to type in code will have ```NotImplementedError()``` as a placeholder. You need to delete this function and replace it with your own code.

## Homework tips 

1. **Start early!** If you run into trouble installing things or importing packages, it’s
best to find those problems well in advance, not the night before your assignment is
due when we cannot help you!

2. **Make sure you back up your work!** At a minimum, do your work in a Dropbox
folder. Better yet, use git, which is well worth your time and effort to learn.

3. **Be careful to follow directions!** Remember that Python is case sensitive. If
you are ask you to define a function called my_function and you define a function
called My_Function, you will not receive full credit. You may want to copy-paste
the function names below to make sure that the functions in your notebook match.

## Error checking

You do not need to do error checking (raising errors, etc.) in your code unless we explicitly ask you to so in a problem.


## Nbgrader

We will be using `nbgrader` to grade your jupyter notebook. You will notice some `read-only` cells in the assignment that contain `assert` statements. These are tests that your code must pass for your solution to be correct. If any of the tests fail, you will get an python error and not get points for that question. 

**Note:** The tests shown not are not comprehensive; additional tests will be used at grading time. You are encouraged to read the problem carefully and verify your code covers all possible cases.

**Be careful:** If a jupyter notebook cell takes longer than `60s` to run, the autograder will not grade it and you will receive zero credit for that question.

## Question 1: Basic SQL (13 pts)
In this problem, you'll interact with a toy SQL database using Python's
built-in `sqlite3` package. Documentation can be found at
<https://docs.python.org/3/library/sqlite3.html>. For this problem,
we'll use a popular toy SQLite database, called Chinook, which
represents a digital music collection. See the documentation at <https://github.com/lerocha/chinook-database/blob/master/README.md>
for a more detailed explanation. We'll use the `chinook.sqlite` file:

In [1]:
import sqlite3
con = sqlite3.connect('chinook.sqlite')
cur = con.cursor()

**1(a)** (2 pt) Load the database using the Python `sqlite3` package. How many tables are in the database? Save the answer in the variable `n_tables`.

In [4]:
# YOUR CODE HERE
cur.execute("SELECT COUNT(*) FROM sqlite_master WHERE type='table';")
n_tables = cur.fetchone()[0]

In [6]:
assert n_tables == 11

**1(b)** (2 pts) What are the names of the tables in the database? Save the answer as
    a set of strings, `table_names`. **Note:** you should write Python `sqlite3`
    code to answer this; don't just look up the answer in the
    documentation!

In [9]:
# YOUR CODE HERE
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = {row[0] for row in cur.fetchall()}

In [11]:
expected = {'Album', 'Genre', 'Playlist', 'PlaylistTrack', 'Employee', 'Customer', 'InvoiceLine', 'Track', 'Artist', 'MediaType', 'Invoice'}
assert table_names == expected

**1(c)** (3 pts) Write a function `albums_starting_with(c)` that takes as an argument a single character `c` and
    returns a list of the primary keys (AlbumIds) of all the albums whose titles
    start with that character. Your function should ignore case, so that
    the inputs "a" and "A" yield the same results. Include error
    checking that raises an error in the event that the input is not a
    single character.


In [14]:
def albums_starting_with(c):
    # YOUR CODE HERE
    if not isinstance(c, str) or len(c) != 1:
        raise Exception("Input must be a single character")
    c = c.lower()
    query = "SELECT AlbumId FROM Album WHERE LOWER(Title) LIKE ?"
    cur.execute(query, (c + '%',))
    album_ids = [row[0] for row in cur.fetchall()]
    return album_ids

In [16]:
res = albums_starting_with('a')
assert type(res) == list
assert len(res) == 32
for i in [10, 14, 15, 24, 26, 29, 74, 167, 319]:
    assert i in res

try:
    albums_starting_with(3)
except Exception:
    pass
else:
    raise Exception("should raise an exception")
    

**1(d)** (3 pts) Write a function `songs_starting_with(c)` that takes as an argument a single character and
    returns a list of the primary keys (TrackIds) of all the **songs** whose album
    names begin with that letter. Again, your function should ignore
    case and perform error checking as in the previous exercise (again ignoring case).
    **Hint:** you'll need a JOIN statement here. Don't forget that you
    can use the `cursor.description` attribute to find out about tables
    and the names of their columns.

In [19]:
def songs_starting_with(c):
    # YOUR CODE HERE
    if not isinstance(c, str) or len(c) != 1:
        raise Exception("Input must be a single character")
    c = c.lower()
    query = """
    SELECT Track.TrackId
    FROM Track
    JOIN Album ON Track.AlbumId = Album.AlbumId
    WHERE LOWER(Album.Title) LIKE ?
    ORDER BY Track.TrackId
    """
    cur.execute(query, (c + '%',))
    track_ids = [row[0] for row in cur.fetchall()]
    return track_ids

In [21]:
res = songs_starting_with('a')
assert type(res) == list
assert len(res) == 369
for i in [85, 86, 87, 331, 332, 333, 923, 924, 925]:
    assert i in res

try:
    songs_starting_with(3)
except Exception:
    pass
else:
    raise Exception("should raise an exception")

try:
    songs_starting_with('res')
except Exception:
    pass
else:
    raise Exception("should raise an exception")

**1(e)** (3 pts) Write a function `cost_of(c)` that takes as an argument a single character and
    returns the cost of buying every song (consider only the songs that were sold - you need to look into InvoiceLine table) whose album begins with that
    letter. This cost should be based on the tracks' unit prices when it was sold, so
    that the cost of buying a set of tracks is simply the sum of the
    unit prices of all the tracks in the set. Again your function should
    ignore case and perform appropriate error checking.


In [59]:
def cost_of(c):
    if len(c) != 1 or not isinstance(c, str):
        raise ValueError("You should input a single character.")
    query = """
        SELECT SUM(Track.UnitPrice)
        FROM Track
        WHERE TrackId IN (
            SELECT InvoiceLine.TrackId
            FROM InvoiceLine
            WHERE InvoiceLine.TrackId IN (
                SELECT Track.TrackId
                FROM Track
                WHERE Track.AlbumId IN (
                    SELECT Album.AlbumId
                    FROM Album
                    WHERE Album.Title LIKE ? COLLATE NOCASE
                )
            )
        )
    """
    
    cur.execute(query, (f"{c}%",))
    result = cur.fetchone()
    return result[0] if result and result[0] is not None else 0

In [61]:
res = cost_of('a')
assert type(res) == float
assert abs(res - 216.82) < 1e-5

try:
    cost_of(3)
except Exception:
    pass
else:
    raise Exception("should raise an exception")

## Problem 2: Building simple models with Pytorch (8 points) 
In this problem, you'll use **Pytorch** to build the loss functions for a pair of commonly-used statistical models. 

We will use variables $X$ and $Y$, which will serve as the predictor (independent variable) and response (dependent variable), respectively. Please use $W$ to denote a parameter that multiplies the predictor, and $b$ to denote a bias parameter (i.e., a parameter that is added).

**2(a)** (4 pts)

In this model, the binary variable $Y$ is distributed as a Bernoulli random variable with success parameter $\sigma(W^T X + b)$, where $\sigma(z) = (1+\exp(-z))^{-1}$ is the logistic function, $X \in R^6$ is the predictor random variable, and $W \in R^6, b \in R$ are the model parameters. 
 
Using **Pytorch** code, implement a class `LogisticRegression` that inherits from `nn.module`. This class should should have two attributes `w` and `b` which should be `nn.parameters` with shapes `(6,1)` and `(1)` respectivelly. 

This class should a method called `forward` that takes in the predictor random variable `x` with shape `(N, 6)`, where `N` is the number of observations, and returns the success parameter (also known as the prediction of our model on $Y$).

**Note:** Please initialize both `w, b` to be __all-one float tensors.__

In [88]:
import torch
import torch.nn as nn
class LogisticRegression(nn.Module):
    # YOUR CODE HERE
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.w = nn.Parameter(torch.ones(6, 1, dtype=torch.float32))
        self.b = nn.Parameter(torch.ones(1, dtype=torch.float32))

    def forward(self, x):
        z = x @ self.w + self.b
        return torch.sigmoid(z)

In [89]:
model = LogisticRegression()
assert type(model.w) == nn.Parameter
assert type(model.b) == nn.Parameter
x = torch.eye(6).float()
y = model.forward(x)
assert y.shape == (6, 1)

**2(b)** (4 pts)

Using **Pytorch** code, write a function called `neg_log` that takes in `y_true` (true value of $Y$) and `y_pred` (predicted value of $Y$) and returns the negative log-likelihood loss function. You can assume that both `y_true` and `y_pred` have shapes `N x 1` where `N` is the number of observations. 
 
 __Hint:__ The loss should be a negative log-likelihood term, summed over all the observations. Remember that $Y$ is Bernoulli distributed which should suggest what the likelihood is. 

In [92]:
def neg_log(y_true, y_pred):
    # YOUR CODE HERE
    loss = -torch.sum(y_true * torch.log(y_pred) + (1 - y_true) * torch.log(1 - y_pred))
    return loss

In [94]:
y_true = torch.tensor([1, 1, 1, 1, 1]).float().view(5, 1)
y_pred = torch.tensor([0.5, 0.5, 0.5, 0.5, 0.5]).float().view(5, 1)
assert 3.4 < neg_log(y_true, y_pred).item() < 3.6